<a href="https://colab.research.google.com/github/Laxmanlee/Toxicity_pred_DM-ML/blob/main/Trials_toxic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install RDKit


In [ ]:
pip install xgboost

In [ ]:
pip install catboost

In [ ]:
from rdkit.Chem import Descriptors

for name,function in Descriptors.descList:
  print(name, function)


# The choice of descriptors depends on the specific toxicity endpoint of interest, 
# as different types of toxicity may be better predicted by different sets of descriptors. 
# However, some commonly used RDKit descriptors for predicting toxicity include molecular weight, number of rotatable bonds, LogP, 
# and a variety of topological, geometric, and electronic descriptors. It's important to note that while descriptors can provide valuable
#  information, they may not be sufficient on their own to accurately predict toxicity, and other factors such as data quality, model selection,
#  and external validation should also be considered.



In [ ]:
# Import required libraries
import pandas as pd
import hashlib
from rdkit import Chem
from rdkit.Chem import Descriptors
from catboost import CatBoostClassifier



# Load the training and test data
train = pd.read_csv("https://raw.githubusercontent.com/Laxmanlee/Toxicity_pred_DM-ML/main/train_II.csv")
test = pd.read_csv("https://raw.githubusercontent.com/Laxmanlee/Toxicity_pred_DM-ML/main/test_II.csv")

# Split the 'Id' column into 'chemical_id' and 'assay_id' columns
train[['chemical_id', 'assay_id']] = train['Id'].apply(lambda x: pd.Series(str(x).split(";")))
test[['chemical_id', 'assay_id']] = test['x'].apply(lambda x: pd.Series(str(x).split(";")))

# Convert the SMILES strings into molecule objects
train['Molecule'] = train['chemical_id'].apply(lambda x: Chem.MolFromSmiles(x))
test['Molecule'] = test['chemical_id'].apply(lambda x: Chem.MolFromSmiles(x))

# Drop rows with invalid SMILES strings
train = train[train['Molecule'].notnull()]
test = test[test['Molecule'].notnull()]




# Train and test the features for the model
train['MolLogP'] = train['Molecule'].apply(lambda x: Descriptors.MolLogP(x))
train['TPSA'] = train['Molecule'].apply(lambda x: Descriptors.TPSA(x))
train['Chi2v'] = train['Molecule'].apply(lambda x: Descriptors.Chi2v(x))
train['MolMR'] = train['Molecule'].apply(lambda x: Descriptors.MolMR(x))
train['Chi1v'] = train['Molecule'].apply(lambda x: Descriptors.Chi1v(x))
train['Chi3v'] = train['Molecule'].apply(lambda x: Descriptors.Chi3v(x))
train['fr_benzene'] = train['Molecule'].apply(lambda x: Descriptors.fr_benzene(x))
train['Chi4v'] = train['Molecule'].apply(lambda x: Descriptors.Chi4v(x))




test['MolLogP'] = test['Molecule'].apply(lambda x: Descriptors.MolLogP(x))
test['TPSA'] = test['Molecule'].apply(lambda x: Descriptors.TPSA(x))
test['Chi2v'] = test['Molecule'].apply(lambda x: Descriptors.Chi2v(x))
test['MolMR'] = test['Molecule'].apply(lambda x: Descriptors.MolMR(x))
test['Chi1v'] = test['Molecule'].apply(lambda x: Descriptors.Chi1v(x))
test['Chi3v'] = test['Molecule'].apply(lambda x: Descriptors.Chi3v(x))
test['fr_benzene'] = test['Molecule'].apply(lambda x: Descriptors.fr_benzene(x))
test['Chi4v'] = test['Molecule'].apply(lambda x: Descriptors.Chi4v(x))




# Gradient boosting classifier
classifier = CatBoostClassifier(n_estimators=1000, learning_rate=0.3, max_depth=10, subsample=0.75,random_state=42)

# Train the model based on the selected features
classifier.fit(train[['assay_id','MolLogP','TPSA','Chi2v','MolMR','Chi1v','Chi3v','fr_benzene','Chi4v']],train['Expected'])

#Predict the outcomes for the test set
test_pred = classifier.predict(test[['assay_id','MolLogP','TPSA','Chi2v','MolMR','Chi1v','Chi3v','fr_benzene','Chi4v']])

#Store the predictions in a data frame
test_predictions = pd.DataFrame({'Id': test['x'], 'Predicted': test_pred})

# print(test.head())
#Write the predictions to a csv file
test_predictions.to_csv('predictions.csv', index=False)

[01:46:32] Explicit valence for atom # 1 Si, 8, is greater than permitted
[01:46:34] Explicit valence for atom # 1 Si, 8, is greater than permitted
[01:46:35] Explicit valence for atom # 1 Si, 8, is greater than permitted


In [ ]:
# Using cross-validation for internal evaluation

# import the required libraries
import pandas as pd
import hashlib
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score
from catboost import CatBoostClassifier


# Load the data
data = pd.read_csv("https://raw.githubusercontent.com/Laxmanlee/Toxicity_pred_DM-ML/main/train_II.csv")

# Split the 'Id' column into 'chemical_id' and 'assay_id' columns
data[['chemical_id', 'assay_id']] = data['Id'].apply(lambda x: pd.Series(str(x).split(";")))

# Convert the SMILES strings into molecule objects
data['Molecule'] = data['chemical_id'].apply(lambda x: Chem.MolFromSmiles(x))

# Drop rows with invalid SMILES strings
data = data[data['Molecule'].notnull()]

# Create new features based on the molecule data

data['MolLogP'] = data['Molecule'].apply(lambda x: Descriptors.MolLogP(x))
data['Chi1v'] = data['Molecule'].apply(lambda x: Descriptors.Chi1v(x))
data['Chi2v'] = data['Molecule'].apply(lambda x: Descriptors.Chi2v(x))
data['Chi3v'] = data['Molecule'].apply(lambda x: Descriptors.Chi3v(x))
data['Chi4v'] = data['Molecule'].apply(lambda x: Descriptors.Chi4v(x))
data['MolMR'] = data['Molecule'].apply(lambda x: Descriptors.MolMR(x))
data['TPSA'] = data['Molecule'].apply(lambda x: Descriptors.TPSA(x))
data['fr_benzene'] = data['Molecule'].apply(lambda x: Descriptors.fr_benzene(x))


# select features and target labels
features = data[['assay_id','MolLogP','TPSA','Chi2v','MolMR','Chi1v','Chi3v','fr_benzene','Chi4v']]
labels = data['Expected']

# Set up the k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Train and evaluate the model using k-fold cross-validation
accuracies = []
for train_index, test_index in kf.split(features):
    # Split data into training and test sets
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]



    # Create CatBoost classifier object and fit on the training data
    classifier = CatBoostClassifier(n_estimators=1000, learning_rate=0.3, max_depth=10, subsample=0.75,random_state=42)
    classifier.fit(X_train, y_train)

    # Predict on the test set and evaluate the model
    y_pred = classifier.predict(X_test)
    accuracy = sum(y_pred == y_test) / len(y_test)
    accuracies.append(accuracy)

# Compute the mean accuracy across all folds
mean_accuracy = sum(accuracies) / k
print(f"Mean accuracy: {mean_accuracy}")

# Calculate F1 score for the test set
print("F1 Score:",f1_score(y_test, y_pred))



